In [1]:
import nltk
import random
import spacy
from spacy import displacy
from collections import Counter
import typing
from typing import List, Dict 
import tqdm
import json
import pandas as pd
import pickle
import string 
import re

In [2]:
file = open("../stored_data/docs.obj",'rb')
cleaned_articles = pickle.load(file)
file.close()

In [3]:
file = open("../stored_data/nouns.obj",'rb')
nouns_per_article = pickle.load(file)
file.close()

In [36]:
# Making a prelimanary count of all the proper nouns I can find. I couldn't track a working spacy method so I converted to string and used native Counter.
# Also made 2 lists, one containing all unique values and 1 containing all values, both as strings.

all_entities = []
counted_entities = Counter()
unique_entities = set()

for key, val in nouns_per_article.items():
    counted_entities += Counter(val)
    all_entities.append(val)

all_entities = [item for sublist in all_entities for item in sublist]
unique_entities = set(all_entities)

In [38]:
# Save the ones with a minimum of 3 links

counted_entities_thresh = Counter({k: c for k, c in counted_entities.items() if c >= 2})
unique_list = [val for val in counted_entities_thresh]
unique_entities = set(unique_list)

In [ ]:
# Calculate similarity of all the entities.

from Levenshtein import ratio

lev_similarity = x = [[] for i in range(len(counted_entities_thresh))]

for idx, val_1 in enumerate(counted_entities_thresh):
    print(idx/len(counted_entities_thresh),end="\r")
    for val_2 in counted_entities_thresh:
        lev_similarity[idx].append(ratio(val_1, val_2))

In [5]:
# we have the articles and the pronouns, I think now we want to find the easiest way to classify them.
# If I can loop up the entity types, I can only look for the ones that don't have an entity...
# Even better if I can loop them up in an existing knowledge base.

import spacy

nlp = spacy.load('en_core_web_lg')

In [51]:
def entity_lookup(doc, uniq):
    recognized_strings = [ent.text for ent in doc.ents]
    matches = set(recognized_strings).intersection(uniq)
    for found in [x for x in matches]:
        uniq.remove(found)
    return uniq

In [60]:
def entity_filter():
    to_be_found = unique_entities
    for key in tqdm.tqdm(list(cleaned_articles.keys())):
        text = ""

        for sent in cleaned_articles[key]:
            text += " " + sent

        doc = nlp(text)

        to_be_found = entity_lookup(doc, to_be_found)
        not_found = to_be_found.copy()

    return not_found

In [61]:
not_found = entity_filter()

100%|██████████| 8399/8399 [05:22<00:00, 26.08it/s]


In [184]:
len(not_found)

2784

In [148]:
done = [x[0] for x in entities_manual]
for idx, x in enumerate(not_found):
    if x not in done:
        print("What type is :")
        print(x)
        print()
        print("X for discard")
        print("S for stop")
        a = input()
        if a == "X":
            pass
        elif a == "S":
            break
        else:
            entities_manual.append([x, a])
    

What type is :
zone

X for discard
S for stop
What type is :
US Europe

X for discard
S for stop
What type is :
GROUPS

X for discard
S for stop
What type is :
Closing

X for discard
S for stop
What type is :
General

X for discard
S for stop
What type is :
Orders

X for discard
S for stop
What type is :
January US

X for discard
S for stop
What type is :
dlr Bally

X for discard
S for stop
What type is :
force

X for discard
S for stop
What type is :
United States Japan West Germany France

X for discard
S for stop
What type is :
Drew

X for discard
S for stop
What type is :
August Australia

X for discard
S for stop
What type is :
PCT PCT

X for discard
S for stop
What type is :
January Shipments

X for discard
S for stop
What type is :
CBI SURVEY

X for discard
S for stop
What type is :
senator

X for discard
S for stop
What type is :
Gulf Kuwait

X for discard
S for stop
What type is :
Franco

X for discard
S for stop
What type is :
Australia Britain

X for discard
S for stop
What 

In [162]:
done = [x[0] for x in entities_manual]
for idx, x in enumerate(not_found):
    if x not in done:
        if "Times".lower() in x.lower():
            entities_manual.append([x, "NEWS"])

done = [x[0] for x in entities_manual]

In [185]:
listr = not_found.copy()

done = [x[0] for x in entities_manual]
for idx, x in enumerate(listr):
    if x not in done:
        if "".lower() in x.lower():
            not_found.remove(x)

done = [x[0] for x in entities_manual]

In [183]:
len(done)

667

In [141]:
x = 110
entities_manual[x:x+100]

[['Overseas Union Bank Ltd', 'COMPANY'],
 ['ltRainbow Corp Ltd Rainbow', 'COMPANY'],
 ['Hoare Govett Asia Ltd', 'COMPANY'],
 ['Ltd', 'COMPANY'],
 ['Deak international Trading Ltd', 'COMPANY'],
 ['CANADA LTD 4TH QTR SHR', 'COMPANY'],
 ['Dome Petroleum Ltd ltDMPMO', 'COMPANY'],
 ['Gas Corp Pty Ltd', 'COMPANY'],
 ['Smelting Co Ltd', 'COMPANY'],
 ['ltsanwa Bank Ltd', 'COMPANY'],
 ['ltI P Sharp Associates Ltd', 'COMPANY'],
 ['Deutsche Babcock AG ltDBCGF', 'COMPANY'],
 ['News Corp Ltd ltNCPAS', 'COMPANY'],
 ['CANBRA FOODS LTD', 'COMPANY'],
 ['Gas Ltd', 'COMPANY'],
 ['Hutchison Whampoa Ltd ltHWHHHK', 'COMPANY'],
 ['Broken Hill Pty Co Ltd ltBRKNS', 'COMPANY'],
 ['Mitsubishi Heavy Industries Ltd ltMITHT MHI', 'COMPANY'],
 ['ltKoba Electronics Co Ltd', 'COMPANY'],
 ['Bond Corp Holdings Ltd ltBONAS', 'COMPANY'],
 ['Dow Chemical Co ltDOW', 'COMPANY'],
 ['Japan Ltd', 'COMPANY'],
 ['Hongkong Land Co Ltd', 'COMPANY'],
 ['Singapore Ltd', 'COMPANY'],
 ['New Zealand Banking Group Ltd ltANZAS', 'COMPANY'

In [186]:
filehandler = open("../stored_data/entities.obj","wb")
pickle.dump(entities_manual, filehandler)
filehandler.close()

In [187]:
file = open("../stored_data/entities.obj",'rb')
entities_manual = pickle.load(file)
file.close()

In [ ]:
"""

yesterday we had a working mvp. Today 

"""